# BentoML Example: Fast AI with Tabular data

BentoML is an open-source framework for machine learning **model serving**, aiming to **bridge the gap between Data Science and DevOps.**

Data Scientists can easily package their models trained with any ML framework using BentoMl and reproduce the model for serving in production. BentoML helps with managing packaged models in the BentoML format, and allows DevOps to deploy them as online API serving endpoints or offline batch inference jobs, on any cloud platform.

Before reading this example project, be sure to check out the [Getting started guide](https://github.com/bentoml/BentoML/blob/master/guides/quick-start/bentoml-quick-start-guide.ipynb) to learn about the basic concepts in BentoML.


This notebook is based on fastai v1's cours v3 lesson 4.  It will train a model that predict salary range base on the data we provided.


![Impression](https://www.google-analytics.com/collect?v=1&tid=UA-112879361-3&cid=555&t=event&ec=fast-ai&ea=fast-ai-salary-range-prediction&dt=fast-ai-salary-range-prediction)

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
!pip install -q -U 'fastai<=1.0.61'

You should consider upgrading via the '/usr/local/anaconda3/envs/dev-py3/bin/python -m pip install --upgrade pip' command.


In [4]:
from fastai.tabular import *

## Prepare Training Data

In [5]:
path = untar_data(URLs.ADULT_SAMPLE)
df = pd.read_csv(path/'adult.csv')

In [6]:
dep_var = 'salary'
cat_names = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race']
cont_names = ['age', 'fnlwgt', 'education-num']
procs = [FillMissing, Categorify, Normalize]

In [7]:
test = TabularList.from_df(df.iloc[800:1000].copy(), path=path, cat_names=cat_names, cont_names=cont_names)

In [8]:
data = (TabularList.from_df(df, path=path, cat_names=cat_names, cont_names=cont_names, procs=procs)
                           .split_by_idx(list(range(800,1000)))
                           .label_from_df(cols=dep_var)
                           .add_test(test)
                           .databunch())

In [9]:
data.show_batch(rows=10)

workclass,education,marital-status,occupation,relationship,race,education-num_na,age,fnlwgt,education-num,target
Private,Assoc-voc,Married-civ-spouse,Adm-clerical,Other-relative,White,False,-1.2891,-0.4098,0.3599,<50k
Private,Some-college,Never-married,Other-service,Own-child,White,False,-1.5090,-1.2826,-0.0312,<50k
Local-gov,12th,Married-civ-spouse,Adm-clerical,Wife,White,False,-0.6294,-1.3385,-0.8135,<50k
Private,HS-grad,Married-civ-spouse,Other-service,Wife,White,False,-0.1163,-0.2701,-0.4224,>=50k
Private,Bachelors,Never-married,Exec-managerial,Not-in-family,White,False,-0.6294,0.1592,1.1422,<50k
?,10th,Divorced,?,Not-in-family,White,False,2.4491,0.7739,-1.5958,<50k
Private,HS-grad,Married-civ-spouse,Transport-moving,Own-child,Other,False,-0.9959,-0.7288,-0.4224,<50k
Self-emp-not-inc,HS-grad,Married-spouse-absent,Craft-repair,Not-in-family,White,False,0.6166,-0.5219,-0.4224,<50k
Local-gov,Assoc-acdm,Married-civ-spouse,Protective-serv,Husband,White,False,-0.7760,-0.1371,0.7511,<50k
Self-emp-not-inc,Bachelors,Married-civ-spouse,Exec-managerial,Own-child,White,False,0.6166,-0.6752,1.1422,<50k


## Model Training

In [10]:
learn = tabular_learner(data, layers=[200,100], metrics=accuracy)

In [11]:
learn.fit(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.362504,0.399898,0.835000,00:04


In [12]:
row = df.iloc[0] # sample input date for testing

learn.predict(row)

(Category tensor(0), tensor(0), tensor([0.5268, 0.4732]))

## Create BentoService for model serving

In [13]:
%%writefile tabular_csv.py

from bentoml import env, api, artifacts, BentoService
from bentoml.frameworks.fastai import FastaiModelArtifact
from bentoml.adapters import DataframeInput


@env(pip_packages=['fastai'])
@artifacts([FastaiModelArtifact('model')])
class FastaiTabularModel(BentoService):
    
    @api(input=DataframeInput(), batch=True)
    def predict(self, df):
        results = []
        for _, row in df.iterrows():       
            prediction = self.artifacts.model.predict(row)
            results.append(prediction[0].obj)
        return results

Overwriting tabular_csv.py


## Save BentoService to file archive

In [14]:
# 1) import the custom BentoService defined above
from tabular_csv import FastaiTabularModel

# 2) `pack` it with required artifacts
svc = FastaiTabularModel()
svc.pack('model', learn)

# 3) save your BentoSerivce
saved_path = svc.save()

[2020-09-22 16:38:31,808] WARNING - Using BentoML installed in `editable` model, the local BentoML repository including all code changes will be packaged together with saved bundle created, under the './bundled_pip_dependencies' directory of the saved bundle.
[2020-09-22 16:38:32,325] INFO - Using default docker base image: `None` specified inBentoML config file or env var. User must make sure that the docker base image either has Python 3.7 or conda installed.
[2020-09-22 16:38:32,327] WARNING - BentoML by default does not include spacy and torchvision package when using FastaiModelArtifact. To make sure BentoML bundle those packages if they are required for your model, either import those packages in BentoService definition file or manually add them via `@env(pip_packages=['torchvision'])` when defining a BentoService
[2020-09-22 16:38:32,329] WARNING - pip package requirement fastai already exist
[2020-09-22 16:38:33,176] INFO - Detected non-PyPI-released BentoML installed, copying 

/usr/local/anaconda3/envs/dev-py3/lib/python3.7/site-packages/setuptools/dist.py:476: UserWarning: Normalizing '0.9.0.pre+3.gcebf2015' to '0.9.0rc0+3.gcebf2015'
  normalized_version,
no previously-included directories found matching 'e2e_tests'
no previously-included directories found matching 'tests'
no previously-included directories found matching 'benchmark'


UPDATING BentoML-0.9.0rc0+3.gcebf2015/bentoml/_version.py
set BentoML-0.9.0rc0+3.gcebf2015/bentoml/_version.py to '0.9.0.pre+3.gcebf2015'
[2020-09-22 16:38:36,944] INFO - BentoService bundle 'FastaiTabularModel:20200922163833_30289D' saved to: /Users/bozhaoyu/bentoml/repository/FastaiTabularModel/20200922163833_30289D


## REST API Model Serving


To start a REST API model server with the BentoService saved above, use the bentoml serve command:

In [15]:
!bentoml serve FastaiTabularModel:latest

[2020-09-22 16:56:10,329] INFO - Getting latest version FastaiTabularModel:20200922163833_30289D
[2020-09-22 16:56:10,330] INFO - Starting BentoML API server in development mode..
[2020-09-22 16:56:10,622] WARNING - Using BentoML installed in `editable` model, the local BentoML repository including all code changes will be packaged together with saved bundle created, under the './bundled_pip_dependencies' directory of the saved bundle.
[2020-09-22 16:56:10,640] WARNING - Saved BentoService bundle version mismatch: loading BentoService bundle create with BentoML version 0.9.0.pre, but loading from BentoML version 0.9.0.pre+3.gcebf2015
[2020-09-22 16:56:11,014] INFO - Using default docker base image: `None` specified inBentoML config file or env var. User must make sure that the docker base image either has Python 3.7 or conda installed.
[2020-09-22 16:56:13,998] WARNING - BentoML by default does not include spacy and torchvision package when using FastaiModelArtifact. To make sure Bento

If you are running this notebook from Google Colab, you can start the dev server with `--run-with-ngrok` option, to gain acccess to the API endpoint via a public endpoint managed by [ngrok](https://ngrok.com/):

In [ ]:
!bentoml serve FastaiTabularModel:latest --run-with-ngrok

### Send prediction requeset to the REST API server

#### JSON Request

```bash
curl -X POST \
  http://localhost:5000/predict \
  -H 'Content-Type: application/json' \
  -d '[{
  "age": 49,
  "workclass": "Private",
  "fnlwgt": 101320,
  "education": "Assoc-acdm",
  "education-num": 12.0,
  "marital-status": "Married-civ-spouse",
  "occupation": "",
  "relationship": "Wift",
  "race": "White",
  "sex": "Female",
  "capital-gain": 0,
  "capital-loss": 1902,
  "hours-per-week": 40,
  "native-country": "United-States",
  "salary": ">=50k"
}]'
```

#### CSV Request

```bash
curl -X POST "http://127.0.0.1:5000/predict" \
    -H "Content-Type: text/csv" \
    --data-binary @test.csv
```

## Containerize model server with Docker


One common way of distributing this model API server for production deployment, is via Docker containers. And BentoML provides a convenient way to do that.

Note that docker is **not available in Google Colab**. You will need to download and run this notebook locally to try out this containerization with docker feature.

If you already have docker configured, simply run the follow command to product a docker container serving the IrisClassifier prediction service created above:

In [16]:
!bentoml containerize FastaiTabularModel:latest

[2020-09-22 16:56:43,434] INFO - Getting latest version FastaiTabularModel:20200922163833_30289D
Found Bento: /Users/bozhaoyu/bentoml/repository/FastaiTabularModel/20200922163833_30289D
[2020-09-22 16:56:43,475] WARNING - Using BentoML installed in `editable` model, the local BentoML repository including all code changes will be packaged together with saved bundle created, under the './bundled_pip_dependencies' directory of the saved bundle.
[2020-09-22 16:56:43,496] WARNING - Saved BentoService bundle version mismatch: loading BentoService bundle create with BentoML version 0.9.0.pre, but loading from BentoML version 0.9.0.pre+3.gcebf2015
Tag not specified, using tag parsed from BentoService: 'fastaitabularmodel:20200922163833_30289D'
Building Docker image fastaitabularmodel:20200922163833_30289D from FastaiTabularModel:latest 
-we in here
processed docker file
(None, None)
root in create archive /Users/bozhaoyu/bentoml/repository/FastaiTabularModel/20200922163833_30289D ['Dockerfile'

/Collecting scipy
|  Downloading scipy-1.5.2-cp37-cp37m-manylinux1_x86_64.whl (25.9 MB)
/Collecting beautifulsoup4
|Collecting nvidia-ml-py3
|Collecting pyyaml
\  Downloading PyYAML-5.3.1.tar.gz (269 kB)
|Collecting matplotlib
\Collecting Pillow
-  Downloading Pillow-7.2.0-cp37-cp37m-manylinux1_x86_64.whl (2.2 MB)
|Collecting torchvision
\Collecting fastprogress>=0.2.1
/Collecting spacy>=2.0.18; python_version < "3.8"
|  Downloading spacy-2.3.2-cp37-cp37m-manylinux1_x86_64.whl (9.9 MB)
/Collecting bottleneck
|  Downloading Bottleneck-1.3.2.tar.gz (88 kB)
  Installing build dependencies: started
\  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
/  Getting requirements to build wheel: finished with status 'done'
|    Preparing wheel metadata: started
/    Preparing wheel metadata: finished with status 'done'
\Collecting numexpr
/Collecting pytz>=2011k
\Collecting future
-Requirement already satisfied: Mako 

-Collecting importlib-metadata>=0.20; python_version < "3.8"
/Collecting zipp>=0.5
|Building wheels for collected packages: nvidia-ml-py3, pyyaml, bottleneck, future
/  Building wheel for nvidia-ml-py3 (setup.py): finished with status 'done'
  Created wheel for nvidia-ml-py3: filename=nvidia_ml_py3-7.352.0-py3-none-any.whl size=19191 sha256=9133f42be2d5c905262136fe2f11fafeee49d48405dfeccf433b1a1f6880596c
  Stored in directory: /tmp/pip-ephem-wheel-cache-3pc2f4g3/wheels/df/99/da/c34f202dc8fd1dffd35e0ecf1a7d7f8374ca05fbcbaf974b83
\  Building wheel for pyyaml (setup.py): finished with status 'done'
  Created wheel for pyyaml: filename=PyYAML-5.3.1-cp37-cp37m-linux_x86_64.whl size=44619 sha256=06b029825047443a06a6d902a0a8aea3043fcda80e5b64aee2d9a3885fdb1686
  Stored in directory: /tmp/pip-ephem-wheel-cache-3pc2f4g3/wheels/5e/03/1e/e1e954795d6f35dfc7b637fe2277bff021303bd9570ecea653
\  Building wheel for bottleneck (PEP 517): finished with status 'done'
  Created wheel for bottleneck: 

|Requirement already satisfied, skipping upgrade: ply<4.0,>=3.4 in /opt/conda/lib/python3.7/site-packages (from thriftpy2>=0.4.0->py-zipkin->BentoML==0.9.0rc0+3.gcebf2015) (3.11)
|  Building wheel for BentoML (PEP 517): finished with status 'done'
  Created wheel for BentoML: filename=BentoML-0.9.0rc0+3.gcebf2015-py3-none-any.whl size=3064091 sha256=762aa6ea85795b1fa82fe6196527c7db0fe6e17da0a48a128cbc2e5b2f846d2d
  Stored in directory: /root/.cache/pip/wheels/a0/45/41/62152db705af4ff47e7a3d6abf6247986eef4aa1b94a58d3b9
Successfully built BentoML
\Installing collected packages: BentoML
  Attempting uninstall: BentoML
    Found existing installation: BentoML 0.9.0rc0
/    Uninstalling BentoML-0.9.0rc0:
\      Successfully uninstalled BentoML-0.9.0rc0
-Successfully installed BentoML-0.9.0rc0+3.gcebf2015
\ ---> e2f758c32fe8
Step 10/15 : ENV PORT 5000
 ---> Running in bcf40c69a9b0
- ---> 3e6f27372f5d
Step 11/15 : EXPOSE $PORT
/ ---> Running in 022d6305af14
| ---> 1466c0426ea9
Step 

In [17]:
!docker run -p 5000:5000 fastaitabularmodel:20200922163833_30289D

[2020-09-23 00:01:08,992] INFO - Starting BentoML API server in production mode..
[2020-09-23 00:01:09,478] INFO - get_gunicorn_num_of_workers: 3, calculated by cpu count
[2020-09-23 00:01:09 +0000] [1] [INFO] Starting gunicorn 20.0.4
[2020-09-23 00:01:09 +0000] [1] [INFO] Listening at: http://0.0.0.0:5000 (1)
[2020-09-23 00:01:09 +0000] [1] [INFO] Using worker: sync
[2020-09-23 00:01:09 +0000] [12] [INFO] Booting worker with pid: 12
[2020-09-23 00:01:09 +0000] [13] [INFO] Booting worker with pid: 13
[2020-09-23 00:01:09 +0000] [14] [INFO] Booting worker with pid: 14
[2020-09-23 00:01:09,734] WARNING - Using BentoML not from official PyPI release. In order to find the same version of BentoML when deploying your BentoService, you must set the 'core/bentoml_deploy_version' config to a http/git location of your BentoML fork, e.g.: 'bentoml_deploy_version = git+https://github.com/{username}/bentoml.git@{branch}'
[2020-09-23 00:01:09,736] WARNING - Using BentoML not from official PyPI relea

## Load saved BentoService

bentoml.load is the API for loading a BentoML packaged model in python:

In [20]:
from bentoml import load

svc = load(saved_path)
print(svc.predict(df.iloc[0:1]))

[2020-09-22 17:01:56,813] WARNING - Saved BentoService bundle version mismatch: loading BentoService bundle create with BentoML version 0.9.0.pre, but loading from BentoML version 0.9.0.pre+3.gcebf2015
[2020-09-22 17:01:56,815] WARNING - Module `tabular_csv` already loaded, using existing imported module.
[2020-09-22 17:01:56,823] WARNING - pip package requirement pandas already exist
[2020-09-22 17:01:56,826] WARNING - BentoML by default does not include spacy and torchvision package when using FastaiModelArtifact. To make sure BentoML bundle those packages if they are required for your model, either import those packages in BentoService definition file or manually add them via `@env(pip_packages=['torchvision'])` when defining a BentoService
[2020-09-22 17:01:56,828] WARNING - pip package requirement torch already exist
[2020-09-22 17:01:56,829] WARNING - pip package requirement fastai already exist
['<50k']


## Launch inference job from CLI

BentoML cli supports loading and running a packaged model from CLI. With the DataframeInput adapter, the CLI command supports reading input Dataframe data from CLI argument or local csv or json files:

In [ ]:
!bentoml run FastaiTabularModel:latest predict \
    --input https://raw.githubusercontent.com/bentoml/gallery/master/fast-ai/salary-range-prediction/test.csv

# Deployment Options

If you are at a small team with limited engineering or DevOps resources, try out automated deployment with BentoML CLI, currently supporting AWS Lambda, AWS SageMaker, and Azure Functions:
- [AWS Lambda Deployment Guide](https://docs.bentoml.org/en/latest/deployment/aws_lambda.html)
- [AWS SageMaker Deployment Guide](https://docs.bentoml.org/en/latest/deployment/aws_sagemaker.html)
- [Azure Functions Deployment Guide](https://docs.bentoml.org/en/latest/deployment/azure_functions.html)

If the cloud platform you are working with is not on the list above, try out these step-by-step guide on manually deploying BentoML packaged model to cloud platforms:
- [AWS ECS Deployment](https://docs.bentoml.org/en/latest/deployment/aws_ecs.html)
- [Google Cloud Run Deployment](https://docs.bentoml.org/en/latest/deployment/google_cloud_run.html)
- [Azure container instance Deployment](https://docs.bentoml.org/en/latest/deployment/azure_container_instance.html)
- [Heroku Deployment](https://docs.bentoml.org/en/latest/deployment/heroku.html)

Lastly, if you have a DevOps or ML Engineering team who's operating a Kubernetes or OpenShift cluster, use the following guides as references for implementating your deployment strategy:
- [Kubernetes Deployment](https://docs.bentoml.org/en/latest/deployment/kubernetes.html)
- [Knative Deployment](https://docs.bentoml.org/en/latest/deployment/knative.html)
- [Kubeflow Deployment](https://docs.bentoml.org/en/latest/deployment/kubeflow.html)
- [KFServing Deployment](https://docs.bentoml.org/en/latest/deployment/kfserving.html)
- [Clipper.ai Deployment Guide](https://docs.bentoml.org/en/latest/deployment/clipper.html)

